# SageMaker Realtime Inference LLama2 13B Chat LMI Model using response streaming


## Set up development environment
Upgrade `pip` and install the latest version of `sagemaker` and `boto3` packages.

In [ ]:
!pip install -Uq pip
!pip install -Uq sagemaker boto3

Restore the `endpoint_name` from the deployment notebook.

In [ ]:
%store -r \
endpoint_name \
bucket \
s3_prefix

In [ ]:
endpoint_name

## Setup Code for Realtime Inference for response streaming

Amazon SageMaker's new `InvokeEndpointWithResponseStream` API allows developers to stream responses back from SageMaker models, which can help to improve customer satisfaction by reducing the perceived latency. This is especially important for applications built with generative AI models, where immediate processing is more important than waiting for the entire response.

The `LineIterator` helper class will parse the response stream received from the inference via `invoke_endpoint_with_response_stream` API made to Llama 2 Chat model running on LMI container.


In [ ]:
import sagemaker
import json
import boto3

sagemaker_runtime = boto3.client('sagemaker-runtime')

In [ ]:
def get_realtime_response_stream(sagemaker_runtime, endpoint_name, payload):
    response_stream = sagemaker_runtime.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name,
        Body=json.dumps(payload), 
        ContentType="application/json",
        CustomAttributes='accept_eula=true'
    )
    return response_stream

The following `print_response_stream` helper function will parse the response stream received from the inference request made via `InvokeEndpointWithResponseStream` API.

In [ ]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils.LineIterator import LineIterator

def print_response_stream(response_stream):
    event_stream = response_stream.get('Body')
    for line in LineIterator(event_stream):
        print(line, end='')

## Prepare Prompt and instructions

To prompt Llama 2 Chat, you need to have following prompt format

```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
```

In [ ]:
def build_llama2_prompt(instructions):
    stop_token = "</s>"
    start_token = "<s>"
    startPrompt = f"{start_token}[INST] "
    endPrompt = " [/INST]"
    conversation = []
    for index, instruction in enumerate(instructions):
        if instruction["role"] == "system" and index == 0:
            conversation.append(f"<<SYS>>\n{instruction['content']}\n<</SYS>>\n\n")
        elif instruction["role"] == "user":
            conversation.append(instruction["content"].strip())
        else:
            conversation.append(f"{endPrompt} {instruction['content'].strip()} {stop_token}{startPrompt}")

    return startPrompt + "".join(conversation) + endPrompt

def get_instructions(user_content):
    
    '''
    Note: We are creating a fresh user content everytime by initializing instructions for every user_content.
    This is to avoid past user_content when you are inferencing multiple times with new ask everytime.
    ''' 
    
    system_content = '''
    You are a friendly and knowledgeable email marketing agent, Mr.MightyMark, working at AnyCompany. 
    Your goal is to send email to subscribers to help them understand the value of the new product and generate excitement for the launch.

    Here are some tips on how to achieve your goal:

    Be Professional. Address each subscriber by name and use a Professional tone.
    Be informative. Explain the key features and benefits of the new product in a clear and concise way.
    Be persuasive. Highlight how the new product can solve the subscriber's problems or improve their lives.

    By following these tips, you can use email marketing to help your company launch a successful software product.
    '''

    instructions = [
        { "role": "system","content": f"{system_content} "},
    ]
    
    instructions.append({"role": "user", "content": f"{user_content}"})
    
    return instructions


## Inference the Llama 2 Chat LMI SageMaker endpoint for Streaming Response

### Inference example 1

In [ ]:
user_ask_1 = f'''
AnyCompany recently announced new service launch named AnyCloud Internet Service.
Write a short email about the product launch with Call to action to Alice Smith, whose email is alice.smith@example.com
Mention the Coupon Code: EARLYB1RD to get 20% for 1st 3 months.
'''

instructions = get_instructions(user_ask_1)
prompt = build_llama2_prompt(instructions)
print(prompt)

In [ ]:
inference_params = {
        "do_sample": True,
        "top_p": 0.6,
        "temperature": 1.0,
        "top_k": 50,
        "max_new_tokens": 512,
        "repetition_penalty": 1.03,
        "stop": ["</s>"],
        "return_full_text": False
    }
payload = {
    "inputs":  prompt,
    "parameters": inference_params,
}

In [ ]:
resp = get_realtime_response_stream(sagemaker_runtime, endpoint_name, payload)
print_response_stream(resp)

### Inference example 2

In [ ]:
user_ask_2 = f'''
AnyCompany recently announced new service launch named AnyCloud Streaming Service.
Write a short email about the product launch with Call to action to Alice Smith, whose email is alice.smith@example.com
Mention the Coupon Code: STREAM2DREAM to get 15% for 1st 6 months.
'''

instructions = get_instructions(user_ask_2)
prompt = build_llama2_prompt(instructions)
inference_params = {
        "do_sample": True,
        "top_p": 0.6,
        "temperature": 0.9,
        "top_k": 50,
        "max_new_tokens": 512,
        "return_full_text": False,
    }
payload = {
    "inputs":  prompt,
    "parameters": inference_params
}
resp = get_realtime_response_stream(sagemaker_runtime, endpoint_name, payload)
print_response_stream(resp)

## Cleanup

Change the value of the variable *cleanup* to `True` to perform clean up activity.

In [ ]:
cleanup = False

In [ ]:
sm_client = boto3.client('sagemaker')
endpoint = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_config_name = endpoint['EndpointConfigName']
endpoint_config = sm_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
model_name = endpoint_config['ProductionVariants'][0]['ModelName']

print(f"""
About to delete the following sagemaker resources:
Endpoint: {endpoint_name}
Endpoint Config: {endpoint_config_name}
Model: {model_name}
""")

In [ ]:
if cleanup:
    # delete endpoint
    sm_client.delete_endpoint(EndpointName=endpoint_name)
    # delete endpoint config
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
    # delete model
    sm_client.delete_model(ModelName=model_name)

In [ ]:
if cleanup:
    # delete model artifacts from s3
    s3 = boto3.resource('s3')
    s3_bucket = s3.Bucket(bucket)
    s3_bucket.objects.filter(Prefix=s3_prefix).delete()